In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [2]:
model_name = "distilbert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=5  # change according to your classes
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import pandas as pd

df = pd.read_csv("/content/training_data.csv",header=None)
df.columns = ["text", "label"]
print(df.columns)


Index(['text', 'label'], dtype='object')


In [4]:


label2id = {label: i for i, label in enumerate(df["label"].unique())}
id2label = {i: label for label, i in label2id.items()}

df["label"] = df["label"].map(label2id)


In [5]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

dataset = dataset.map(tokenize, batched=True)
dataset = dataset.rename_column("label", "labels")
dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/120001 [00:00<?, ? examples/s]

In [7]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)
print("works")
trainer.train()


wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

works
 b6af57b92527b72d43162bb325d5c561c1f5b6d4


wandb: WARNING Invalid choice
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alikhizer1947 (alikhizer1947-national-university-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.435400
1000,0.299000
1500,0.272600
2000,0.248800
2500,0.232000
3000,0.232900
3500,0.230000
4000,0.228700
4500,0.224300
5000,0.219300


TrainOutput(global_step=15002, training_loss=0.19679273760728186, metrics={'train_runtime': 6347.8021, 'train_samples_per_second': 37.809, 'train_steps_per_second': 2.363, 'total_flos': 1.589707076568576e+16, 'train_loss': 0.19679273760728186, 'epoch': 2.0})

In [8]:
# -------------------------------
# Save the trained model and tokenizer
# -------------------------------

output_dir = "./trained_model"

# Save the model
trainer.model.save_pretrained(output_dir)

# Save the tokenizer
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")


Model and tokenizer saved to ./trained_model


In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# # Load the saved model and tokenizer
# model_path = "./trained_model"
# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForSequenceClassification.from_pretrained(model_path)

# # Create the pipeline for inference
# classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)


In [9]:
from sklearn.metrics import accuracy_score, f1_score

preds = trainer.predict(dataset)
y_pred = preds.predictions.argmax(axis=1)
y_true = preds.label_ids

print("Accuracy:", accuracy_score(y_true, y_pred))
print("F1 Score:", f1_score(y_true, y_pred, average="weighted"))


Accuracy: 0.9692335897200857
F1 Score: 0.9692469052323798


In [10]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer
)


Device set to use cuda:0


In [ ]:
import gradio as gr

# Your id2label mapping
id2label = {
    0: 'World',
    1: 'Sports',
    2: 'Business',
    3: 'Sci/Tech',
    4: 'Crime'
}

def predict_text(text):
    # classifier returns a list of dicts [{'label': 'LABEL_0', 'score': 0.99}]
    result = classifier(text)[0]  # get top prediction
    # Convert 'LABEL_0' → 0
    predicted_id = int(result['label'].split('_')[-1])
    # Map to original category
    original_label = id2label[predicted_id]
    # Return nicely formatted string
    return f"Predicted Category: {original_label}\nConfidence: {result['score']:.4f}"

# Create Gradio interface
iface = gr.Interface(
    fn=predict_text,
    inputs=gr.Textbox(lines=5, label="Enter text for classification"),
    outputs=gr.Textbox(label="Prediction"),
    title="News Text Classification",
    description="Enter a news text and get its predicted category."
)

iface.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4d68430a9f7de4b2f3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr

def predict_text(text):
    # The classifier pipeline returns a list of dictionaries, e.g., [{'label': 'LABEL_0', 'score': 0.99}]
    result = classifier(text)[0]
    # Extract the predicted label index (e.g., 0 from 'LABEL_0')
    predicted_id_str = result['label']
    predicted_id = int(predicted_id_str.split('_')[-1])

    # Map the predicted ID back to its original label using id2label
    original_label = id2label[predicted_id]

    return f"Predicted Class: {original_label} (Confidence: {result['score']:.4f})"

# Create the Gradio interface
iface = gr.Interface(
    fn=predict_text,
    inputs=gr.Textbox(lines=5, label="Enter text for classification"),
    outputs=gr.Textbox(label="Prediction"),
    title="Text Classification Model",
    description="Enter text to classify it into one of the predefined categories."
)

# Launch the interface
iface.launch(debug=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')